In [1]:
from ultralytics import YOLO
import os
import xml.etree.ElementTree as ET

In [2]:
#model = YOLO('yolov8n.pt')
#model.predict(
#   source='https://media.roboflow.com/notebooks/examples/dog.jpeg',
#   conf=0.25
#)

In [3]:
# !yolo predict model=yolov8n.pt source='/Users/erik/repos/ComputerVisionProject/img.png'

In [4]:
def does_file_exist(file_path):
    return os.path.exists(file_path)

In [5]:
def does_xml_file_contain_track_element(file_path):
    with open(file_path, 'r') as file:
        data = file.read()
        return '<track' in data

In [6]:
def rename_folder_space_to_underscore(folder_path):
    os.rename(folder_path, folder_path.replace(' ', '_'))

In [7]:
for folder in os.listdir('/Users/erik/repos/ComputerVisionProject/data/Annotation'):
    folder_path = f'/Users/erik/repos/ComputerVisionProject/data/Annotation/{folder}'
    if os.path.isdir(folder_path):
        rename_folder_space_to_underscore(folder_path)

In [8]:
# Create output to be pasted into yaml file
# - task_fish_detection-2024_02_05_07_51_04-cvat_for_video_1.1
for folder in os.listdir('/Users/erik/repos/ComputerVisionProject/data/Annotation'):
    folder_path = f'/Users/erik/repos/ComputerVisionProject/data/Annotation/{folder}'
    if os.path.isdir(folder_path):
        print(f'- {folder}')

- task_fish_detection-2024_01_28_18_07_53-cvat_for_video_1.1


In [9]:
# Create two folders: one with fish and one without fish
if not os.path.exists('/Users/erik/repos/ComputerVisionProject/data/fish'):
    os.mkdir('/Users/erik/repos/ComputerVisionProject/data/fish')
if not os.path.exists('/Users/erik/repos/ComputerVisionProject/data/no_fish'):
    os.mkdir('/Users/erik/repos/ComputerVisionProject/data/no_fish')

In [10]:
# Move folders with fish to fish folder and folders without fish to no_fish folder
# for folder in os.listdir('/Users/erik/repos/ComputerVisionProject/data/Annotation'):
#     folder_path = f'/Users/erik/repos/ComputerVisionProject/data/Annotation/{folder}'
#     if os.path.isdir(folder_path):
#         if does_file_exist(f'{folder_path}/annotations.xml') and does_xml_file_contain_track_element(
#                 f'{folder_path}/annotations.xml'):
#             os.rename(folder_path, f'/Users/erik/repos/ComputerVisionProject/data/fish/{folder}')
#         else:
#             os.rename(folder_path, f'/Users/erik/repos/ComputerVisionProject/data/no_fish/{folder}')

In [11]:
## Example xml:
# <track id="0" label="SEACHAR" source="manual">
#    <box frame="71" outside="0" occluded="0" keyframe="1" xtl="22.83" ytl="718.40" xbr="140.30" ybr="868.50" z_order="0">
#    </box>
#    <box frame="72" outside="0" occluded="0" keyframe="0" xtl="22.31" ytl="714.45" xbr="158.30" ybr="866.73" z_order="0">
#    </box>
#    ...
#    <box frame="179" outside="0" occluded="0" keyframe="0" xtl="12.53" ytl="510.70" xbr="99.20" ybr="675.10" z_order="0">
#    </box>
#    <box frame="180" outside="1" occluded="0" keyframe="1" xtl="12.53" ytl="510.70" xbr="99.20" ybr="675.10" z_order="0">
#    </box>
# </track>

In [12]:
def get_file_path_from_frame_number_and_folder_path(folder_path, frame_number):
    # Make frame_number be 6 digit with leading zeros
    frame_number = str(frame_number).zfill(6)
    # folder -> images -> frame_{frame_number}.PNG
    file_path = f'{folder_path}/images/frame_{frame_number}.PNG'
    if os.path.exists(file_path):
        print("Found file path: ", file_path)
        return file_path
    else:
        print("Did not find file path: ", file_path)
        return None

In [13]:
def get_images_with_fish_from_folder_path(folder_path):
    images_with_fish = []
    annotation_file_path = folder_path + '/annotations.xml'  # Unsafe

    try:
        tree = ET.parse(annotation_file_path)
        root = tree.getroot()

        for track in root.findall('track'):
            for box in track.findall('box'):
                frame_number = int(box.get('frame'))
                file_path = get_file_path_from_frame_number_and_folder_path(folder_path, frame_number)
                if file_path is not None:
                    images_with_fish.append(file_path)

    except FileNotFoundError:
        print("Could not find annotation file.")
    except Exception as e:
        print(f"Error: {e}")

    return images_with_fish

In [14]:
# Move images with fish to fish folder and remove images without fish
for folder in os.listdir('/Users/erik/repos/ComputerVisionProject/data/Annotation'):
    folder_path = f'/Users/erik/repos/ComputerVisionProject/data/Annotation/{folder}'
    if os.path.isdir(folder_path):
        images_file_paths_with_fish = get_images_with_fish_from_folder_path(folder_path)
        for image_path in images_file_paths_with_fish:
            os.rename(image_path, f'/Users/erik/repos/ComputerVisionProject/data/fish/{image_path.split("/")[-1]}')

Found file path:  /Users/erik/repos/ComputerVisionProject/data/Annotation/task_fish_detection-2024_01_28_18_07_53-cvat_for_video_1.1/images/frame_000071.PNG
Found file path:  /Users/erik/repos/ComputerVisionProject/data/Annotation/task_fish_detection-2024_01_28_18_07_53-cvat_for_video_1.1/images/frame_000072.PNG
Found file path:  /Users/erik/repos/ComputerVisionProject/data/Annotation/task_fish_detection-2024_01_28_18_07_53-cvat_for_video_1.1/images/frame_000073.PNG
Found file path:  /Users/erik/repos/ComputerVisionProject/data/Annotation/task_fish_detection-2024_01_28_18_07_53-cvat_for_video_1.1/images/frame_000074.PNG
Found file path:  /Users/erik/repos/ComputerVisionProject/data/Annotation/task_fish_detection-2024_01_28_18_07_53-cvat_for_video_1.1/images/frame_000075.PNG
Found file path:  /Users/erik/repos/ComputerVisionProject/data/Annotation/task_fish_detection-2024_01_28_18_07_53-cvat_for_video_1.1/images/frame_000076.PNG
Found file path:  /Users/erik/repos/ComputerVisionProject/

In [15]:
# Move remaining images to no_fish folder
for folder in os.listdir('/Users/erik/repos/ComputerVisionProject/data/Annotation'):
    folder_path = f'/Users/erik/repos/ComputerVisionProject/data/Annotation/{folder}'
    if os.path.isdir(folder_path):
        for image in os.listdir(f'{folder_path}/images'):
            os.rename(f'{folder_path}/images/{image}', f'/Users/erik/repos/ComputerVisionProject/data/no_fish/{image}')

In [22]:
model = YOLO("yolov8n.yaml")

# Train the model
model.train(data="../nemoV1.yaml", epochs=1)

New https://pypi.org/project/ultralytics/8.1.42 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.41 🚀 Python-3.10.6 torch-2.2.2 CPU (Intel Core(TM) i5-1038NG7 2.00GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=../nemoV1.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=

FileNotFoundError: [34m[1mtrain: [0mError loading data from ['/Users/erik/repos/data/fish', '/Users/erik/repos/data/no_fish']
See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.